## Deliverable 3. Create a Travel Itinerary Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [2]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("WeatherPy_vacation.csv").set_index("City_ID")
vacation_df.drop(columns="City_ID.1",inplace=True)
vacation_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
City_ID,,,,,,,,,,,
0,Ixtapa,20.7000,-105.2000,75.22,89,100,5.01,MX,2022-01-19 23:50:07,Hotel Plaza Providencia,4.4
1,Karaton,-6.3424,106.1045,79.95,93,97,5.17,ID,2022-01-19 23:50:07,Parkir Mobil Wisata CAS Water Park,4.3
2,Ushuaia,-54.8000,-68.3000,53.26,58,0,17.27,AR,2022-01-19 23:45:13,Albatross Hotel,4.3
3,Busselton,-33.6500,115.3333,78.04,52,2,9.98,AU,2022-01-19 23:50:08,Observatory Guest House,4.5
4,Pacific Grove,36.6177,-121.9166,65.43,75,0,1.01,US,2022-01-19 23:50:08,Lovers Point Inn,3.7


In [3]:
country_counts_df = vacation_df.groupby('Country').count().sort_values('City', ascending = False)[['City']]
country_counts_df.head()

,City
Country,
US,40
RU,31
BR,26
AU,23
ID,20


In [4]:
def check_hotel_count(country):
    return country_counts_df.loc[country,:].values[0]
check_hotel_count('GA')

4

In [5]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>Rating</dt><dd>{Hotel Rating}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [6]:
# 4a. Add a marker layer for each city to the map. 
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [31]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
import dataframe_image as dfi

def get_random_itinerary(number, country):
    if (country == ""):
        #print(country_counts_df.loc[(country_counts_df['City_ID'] >= int(number))])
        country = country_counts_df.loc[(country_counts_df['City'] >= int(number))].sample(1).index[0]
        print(f"We chose the country {country} at random for you!")
    
    if (country != ""):
        hotel_count = check_hotel_count(country)
        if (hotel_count >= number):
            temp = vacation_df.loc[(vacation_df['Country']==country)]
            #print(temp)
            itinerary_df = temp.sample(number)
            dfi.export(itinerary_df, "{country}_random_itinerary.png")
            return itinerary_df
            
        else:
            user_input = input(f"Sorry, we only found {hotel_count} Hotels in {country}, (y) to continue with this country, otherwise to exit")
            if (user_input == "y"):
                get_random_itinerary(hotel_count, country)
                return
            else:
                return      

In [29]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.
vacation = get_random_itinerary(4,"")

vacation_start = vacation_df.iloc[0,:]
vacation_end = vacation_df.iloc[0,:]
vacation_stop1 = vacation_df.iloc[1,:]
vacation_stop2 = vacation_df.iloc[2,:]
vacation_stop3 = vacation_df.iloc[3,:]

vacation

We chose the country CD at random for you!


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
City_ID,,,,,,,,,,,
535,Mbandaka,0.0487,18.2603,71.96,63,100,2.10,CD,2022-01-20 00:03:14,Guest House Epervier,5.0
111,Bosobolo,4.1833,19.9000,77.52,47,79,1.99,CD,2022-01-19 23:52:35,NaN,NaN
406,Kikwit,-5.0410,18.8162,69.26,96,100,0.94,CD,2022-01-19 23:59:44,HOTEL PLACE YALABA,NaN
688,Gemena,3.2500,19.7667,71.92,42,99,1.63,CD,2022-01-20 00:06:49,Hôtel Goshen,2.0


In [8]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
def get_lat_lng(df):
    return df[['Lat','Lng']].to_numpy()
get_lat_lng(get_random_itinerary(4,""))

We chose the country NO at random for you!


array([[63.7256,  8.834 ],
       [61.9875,  5.1352],
       [67.667 , 12.6934],
       [70.8578, 29.0864]])

In [9]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".
import gmaps.datasets

def create_directions_layer(coords, style):
    fig = gmaps.figure()
    
    start = coords[0]
    
    if style != "one-way":
        style = "loop"
        end = start
        my_directions_layer = gmaps.directions_layer(start,end, waypoints=coords[1:], travel_mode='DRIVING', show_markers=False)
    else:
        end = coords[-1]
        my_directions_layer = gmaps.directions_layer(start, end, waypoints=coords[1:-1], travel_mode='DRIVING', show_markers=False)
    
    return my_directions_layer

In [10]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
def format_df_markers(df):
    info_box_template = """
    <dl>
    <dt>Hotel Name</dt><dd>{Hotel Name}</dd>
    <dt>Rating</dt><dd>{Hotel Rating}</dd>
    <dt>City</dt><dd>{City}</dd>
    <dt>Country</dt><dd>{Country}</dd>
    <dt>Max Temp</dt><dd>{Max Temp} °F</dd>
    </dl>
    """

    # 10a Get the data from each row and add it to the formatting template and store the data in a list.
    formatted_data = [info_box_template.format(**row) for index, row in df.iterrows()]
    return formatted_data

    # 10b. Get the latitude and longitude from each row and store in a new DataFrame.
def draw_itinerary(df,style):
    locations = df[["Lat", "Lng"]]
    
    hotel_info = format_df_markers(df)
    
    # 11a. Add a marker layer for each city to the map.
    max_temp = df["Max Temp"]
    fig = gmaps.figure()
    directions_layer = create_directions_layer(get_lat_lng(df),style)
    marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
    heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
                 max_intensity=300, point_radius=4)
    fig.add_layer(directions_layer)
    fig.add_layer(heat_layer)
    fig.add_layer(marker_layer)    
    
# 11b. Display the figure
    return fig

In [11]:
get_random_itinerary(4,"")

We chose the country CL at random for you!


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Rating
City_ID,,,,,,,,,,,
459,Ancud,-41.8697,-73.8203,66.22,79,26,12.73,CL,2022-01-20 00:00:57,Hotel Arena Gruesa,4.1
172,Castro,-42.4721,-73.7732,69.28,49,0,6.91,CL,2022-01-19 23:53:49,Hotel Esmeralda,4.3
203,Coquimbo,-29.9533,-71.3436,65.93,63,0,10.36,CL,2022-01-19 23:53:41,Hostel Del Puerto,4.3
98,Copiapó,-27.3667,-70.3333,64.09,53,0,5.23,CL,2022-01-19 23:52:31,San Francisco de La Selva,4.0


In [12]:
draw_itinerary(get_random_itinerary(4,""),"one-way")

We chose the country NE at random for you!


Figure(layout=FigureLayout(height='420px'))

In [20]:
draw_itinerary(get_random_itinerary(6,"US"),"loop")

Figure(layout=FigureLayout(height='420px'))

In [14]:
import googlemaps
import simplejson
import json
import numpy as np

gdistance = googlemaps.Client(key = g_key)

def get_distance_matrix(coords):
    durations = []
    result = gdistance.distance_matrix(coords,coords, mode='driving')
    
    #print(json.dumps(result, indent = 4))
    
    df = pd.DataFrame(index=result['origin_addresses'],
                      columns=result['destination_addresses'])
    
    for i in range(len(coords)):
        for j in range(len(coords)):
            try:
                #This is the duration of the travel time in seconds. Divide by 3600 for H:M:S
                df.iloc[i,j] = result["rows"][i]["elements"][j]["duration"]["value"]
            except (KeyError):
                df.iloc[i,j] = np.nan    
    return df
    
    #result = gdistance.distance_matrix(origin, destination, mode='driving')["rows"][0]["elements"][0]["duration"]["value"]  
    #duration = duration/3600
    #durations.append(duration)
    
    #return durations

In [15]:
from sys import maxsize
from itertools import permutations
 
# implementation of traveling Salesman Problem
def travellingSalesmanProblem(graph, start, style):
    if start == "":
        start = 0
        
    # store minimum weight Hamiltonian Cycle
    min_path = maxsize
    path_length = 0
    best_walk = ""
    short_path = ()
    
    for perm in permutations(range(len(graph))):
        if perm[0] != start:
            continue
        log=""
        path_length = 0
        log +=f"{perm}\n"
        for i in perm:
            if i != perm[-1]:
                path_length += graph[i][perm[perm.index(i)+1]]
                log += f"Walking from node {i} to node {perm[perm.index(i)+1]} at a cost of {graph[i][perm[perm.index(i)+1]]}, path length is now: {path_length}\n"
            elif (style == "loop"):
                path_length += graph[i][perm[0]]
                log += f"Walking from node {i} to node {perm[0]} at a cost of {graph[i][perm[0]]}, path length is now: {path_length}\n"
        # print(log)
        if path_length < min_path:
            min_path = path_length
            short_path = perm
            best_walk = log
    print(f"{best_walk}")
    return [short_path,min_path]

# matrix representation of graph
graph = [[0, 10, 15, 20], [10, 0, 35, 25],
        [15, 35, 0, 30], [20, 25, 30, 0]]
travellingSalesmanProblem(graph,"","")

(0, 1, 3, 2)
Walking from node 0 to node 1 at a cost of 10, path length is now: 10
Walking from node 1 to node 3 at a cost of 25, path length is now: 35
Walking from node 3 to node 2 at a cost of 30, path length is now: 65



[(0, 1, 3, 2), 65]

In [16]:
#Here are other datasets easily available from gmaps
gmaps.datasets.list_datasets()

dict_keys(['taxi_rides', 'earthquakes', 'acled_africa', 'acled_africa_by_year', 'london_congestion_zone', 'nuclear_plants', 'starbucks_kfc_uk', 'gini'])

In [17]:
def get_random_sorted_itinerary(number, country, style):
    itinerary = get_random_itinerary(number,country)
    #print(itinerary)
    new_order = list(travellingSalesmanProblem(get_distance_matrix(get_lat_lng(itinerary)).values,"",style)[0])
    #print(new_order)
    sorted_itinerary = itinerary.copy()
    for i in range(len(itinerary)):
        sorted_itinerary.iloc[i,:] = itinerary.iloc[new_order[i],:]
    #print(sorted_itinerary)
    #return sorted_itinerary
    return draw_itinerary(itinerary,style);

In [24]:
get_random_sorted_itinerary(5,"","loop")

(0, 3, 1, 4, 2)
Walking from node 0 to node 3 at a cost of 24331, path length is now: 24331
Walking from node 3 to node 1 at a cost of 126438, path length is now: 150769
Walking from node 1 to node 4 at a cost of 60222, path length is now: 210991
Walking from node 4 to node 2 at a cost of 79417, path length is now: 290408
Walking from node 2 to node 0 at a cost of 48088, path length is now: 338496



Figure(layout=FigureLayout(height='420px'))